In [1]:
import pandas as pd
import numpy as np

In [2]:
trainX=pd.read_csv('train.csv')

In [3]:
trainX.head()

,PID,Date,Event
0,1028890,201101,2186
1,1028890,201101,7087
2,1028890,201101,4848
3,1028890,201101,2214
4,1028890,201102,7087


In [4]:
trainX.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 766787 entries, 0 to 766786
Data columns (total 3 columns):
PID      766787 non-null int64
Date     766787 non-null int64
Event    766787 non-null object
dtypes: int64(2), object(1)
memory usage: 17.6+ MB


In [5]:
#trainX = trainX.values

In [6]:
train_1 =trainX.groupby(['PID','Date']).agg(lambda x:x.value_counts().index[0]).reset_index()

In [7]:
train_1.head()

,PID,Date,Event
0,1000001,201101,3263
1,1000001,201102,2632
2,1000001,201103,3257
3,1000001,201104,2533
4,1000001,201105,3074


In [8]:
train_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70152 entries, 0 to 70151
Data columns (total 3 columns):
PID      70152 non-null int64
Date     70152 non-null int64
Event    70152 non-null object
dtypes: int64(2), object(1)
memory usage: 1.6+ MB


In [9]:
trainX1=pd.get_dummies(train_1['PID'])

In [10]:
trainX2=pd.DataFrame()
trainX2['Year'] = ((train_1['Date']%1000)/100).astype(np.uint8)
trainX2['Month'] = (train_1['Date']%100).astype(np.uint8)

In [11]:
trainX1=trainX1.values

In [12]:
trainX2 = trainX2.values

In [13]:
trainX1=np.concatenate((trainX1,trainX2),axis=1)

In [14]:
trainX1.shape

(70152, 3002)

In [29]:
trainX1

array([[[ 1],
        [ 0],
        [ 0],
        ..., 
        [ 0],
        [ 1],
        [ 1]],

       [[ 1],
        [ 0],
        [ 0],
        ..., 
        [ 0],
        [ 1],
        [ 2]],

       [[ 1],
        [ 0],
        [ 0],
        ..., 
        [ 0],
        [ 1],
        [ 3]],

       ..., 
       [[ 0],
        [ 0],
        [ 0],
        ..., 
        [ 1],
        [ 3],
        [ 6]],

       [[ 0],
        [ 0],
        [ 0],
        ..., 
        [ 1],
        [ 3],
        [ 7]],

       [[ 0],
        [ 0],
        [ 0],
        ..., 
        [ 1],
        [ 3],
        [11]]], dtype=uint8)

In [15]:
trainX1 = np.reshape(trainX1,(trainX1.shape[0],3002,1))

In [16]:
trainY= pd.get_dummies(train_1['Event'])

In [17]:
trainY.shape

(70152, 3649)

In [18]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'
import tensorflow as tf
from keras.layers import Dense,LSTM
from keras.models import Sequential

Using TensorFlow backend.


In [78]:
reg = Sequential()

In [79]:
reg.add(LSTM(units =40,activation = 'sigmoid',input_shape = (None , 1)))

In [80]:
reg.add(Dense(units = 3000,kernel_initializer = 'uniform'))
#reg.add(Dense(units = 6472 , kernel_initializer = 'uniform'))
reg.add(Dense(units=3649,kernel_initializer='uniform', activation='linear'))
reg.compile(optimizer='adam',loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [ ]:
reg.fit(trainX1,trainY.values,batch_size = 100,epochs = 2)

Epoch 1/2
28300/70152 [===========>..................] - ETA: 1144s - loss: 11.7082 - categorical_accuracy: 0.0353

In [ ]:
reg.save('LSTM_Final_changed_v2.h5')

In [30]:
testX2 = pd.DataFrame()
testX1=pd.get_dummies(train_1['PID'].unique())


In [31]:
testX1.shape

(3000, 3000)

In [32]:
testX2.shape

(0, 0)

In [35]:
zero_data = np.zeros(shape=(3000,2))
d = pd.DataFrame(zero_data, columns=['Year','Month'])

In [36]:
d.head()

,Year,Month
0,0.0,0.0
1,0.0,0.0
2,0.0,0.0
3,0.0,0.0
4,0.0,0.0


In [37]:
d.shape

(3000, 2)

In [38]:
d['Month']=01

In [39]:
d.head()

,Year,Month
0,0.0,1
1,0.0,1
2,0.0,1
3,0.0,1
4,0.0,1


In [40]:
d['Year']=14

In [41]:
d.head()

,Year,Month
0,14,1
1,14,1
2,14,1
3,14,1
4,14,1


In [42]:
testX2 = d.values.astype(np.uint8)

In [43]:
testX2.shape

(3000, 2)

In [44]:
testX1 = testX1.values

In [45]:
testX1=np.concatenate((testX1,testX2),axis=1)

In [46]:
testX1.shape

(3000, 3002)

In [47]:
testX1 = np.reshape(testX1,(testX1.shape[0],3002,1))

In [48]:
testX1.shape

(3000, 3002, 1)

In [50]:
y_pred1 = reg.predict(testX1,batch_size = 100)

In [65]:
yy_pred1=y_pred1*10
pred1 = pd.DataFrame(data=yy_pred1)
pred1['PID'] = train_1['PID'].unique()
submit_pred1 = pred1.loc[:,pred1.columns != 'PID'].apply(lambda x: pd.Series(x.sort_values(ascending=False).iloc[:1].index, index=['Event'+str(x) for x in range(1,2)]),axis=1).reset_index()
submit_pred1.drop('index',inplace=True, axis=1)
submit_pred1.head()


,Event1
0,1463
1,1463
2,1463
3,1463
4,1463


In [53]:
y_pred1.max()

0.18375376

In [55]:
y_pred1.min()

1.4838499e-12

In [64]:
((y_pred1*10).max().round())

2.0

In [67]:
y_new_pred = pred1.idxmax(1)

In [62]:
(y_pred1*10).min().round()

0.0

In [69]:
pred1_df = pd.DataFrame(data=y_pred1*10)
pred1_df = pred1_df.idxmax(1)
pred1_df.head()

0    1463
1    1463
2    1463
3    1463
4    1463
dtype: int64

In [70]:
zero_data = np.zeros(shape=(3000,2))
d = pd.DataFrame(zero_data, columns=['Year','Month'])
d['Month']=02
d['Year']=14
testX22 = d.values.astype(np.uint8)
testX21=pd.get_dummies(train_1['PID'].unique())
testX21 = testX21.values
testX21=np.concatenate((testX21,testX22),axis=1)
testX21 = np.reshape(testX21,(testX21.shape[0],3002,1))

In [73]:
y_pred2 = reg.predict(testX21,batch_size = 100)
pred2_df = pd.DataFrame(data=y_pred2)
pred2_df = pred2_df.idxmax(1)
pred2_df.head()


0    1463
1    1463
2    1463
3    1463
4    1463
dtype: int64